In [1]:
'''
    自行操作 的 豆瓣实训
'''
import csv
import requests
from bs4 import BeautifulSoup
import pymysql
import re

def get_one_page(url):
    header = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0',
        'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'cookie':'ll="118318"; bid=z6gJQ-g_VbA; douban-fav-remind=1; _ga=GA1.1.152072664.1728867251; _ga_RXNMP372GL=GS1.1.1728871022.2.0.1728871022.60.0.0; __utmz=30149280.1729470866.7.6.utmcsr=cn.bing.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmv=30149280.28528; ap_v=0,6.0; _pk_id.100001.3ac3=b11897d958dfba39.1742456607.; _pk_ses.100001.3ac3=1; __utma=30149280.148866276.1726189819.1736752951.1742456607.22; __utma=81379588.152072664.1728867251.1742456607.1742456607.1; __utmz=81379588.1742456607.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); dbcl2="285281789:czKJVYE1R0c"; ck=7OEk; frodotk_db="3c2301308a87d870b46903919f978950"; push_noty_num=0; push_doumail_num=0; __utmc=30149280; __utmt=1; __utmt_douban=1; __utmb=30149280.13.10.1742456607; __utmc=81379588; __utmb=81379588.11.10.1742456607'
    }

    r = requests.get(url,headers = header)
    r.encoding = r.apparent_encoding    #设置编码格式
    demo = r.text
    return demo


def parse_one_page(html):
    soup = BeautifulSoup(html,'html.parser')
    Books = soup.select('#content > div.grid-16-8.clearfix > div.article > ul > li')

    Book_data = []
    for book in Books:
        #书名
        Book_name = book.select('div.media__body > h2 > a')[0].string
        
        #作者 但是带出版社？怎么分割
        Book_author = book.select('div.media__body > p.subject-abstract.color-gray')[0].string.strip().split(' / ')
        
        #作者
        writer = Book_author[0].strip()

        #出版时间
        date = '未知'
        for i in Book_author:
            if re.search(r'\d{4}-\d{1,2}(?:-\d{1,2})?',i):
                date = i.strip()
                break
        

        #出版社
        company = '未知'
        for part in Book_author:
            if re.search(r'.*(?:出版|书店|书局|书馆|编选).*',part):
                company = part.strip()
                break


        '''
        #出版时间
        #date = Book_author[1].strip()   #但是匹配的不准确

        #出版社
        #company = Book_author[2].strip()    #但是匹配的不准确
        '''

        #评分
        Book_elem = book.select('div.media__body > p.clearfix.w250.subject-rating > span.font-small.fleft')
        Book_rank = Book_elem[0].string if Book_elem and Book_elem[0].string else '未评分'
        
        #评分人数
        Book_rankPerson = book.find('span',class_='fleft ml8 color-gray').text.replace('(','').replace(')','')

        #图书链接
        Book_src = book.select('div.media__body > h2 > a')[0]['href']
        

        Book_data.append([Book_name,writer,company,date,Book_rank,Book_rankPerson,Book_src])

        #print(Book_name,writer,company,date,Book_rank,Book_rankPerson,Book_src)
        
        
    return Book_data
        
        


def write_to_csv(content,path):
    with open(path,'a',newline='',encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            writer.writerow(['书名','作者','出版社','出版时间','评分','评分人数','图书链接'])
        writer.writerows(content)   #写入数据


def write_to_table(content):
    #1、创建数据库链接对象connect
    db = pymysql.connect(
        host='localhost',
        port=3306,
        user='root',
        passwd='1024',
        db='douban_book',
        charset='utf8'
    )

    #2、获取游标对象 cursor
    cursor = db.cursor()

    #3、执行SQL语句（核心步骤：建表、增删改查）
    

    create_table = """
    create table if not exists book1(
        book_name varchar(100) not null primary key,
        book_author varchar(100) not null,
        book_company varchar(100) not null,
        book_date varchar(100) not null,
        book_rank varchar(100) not null,
        book_person varchar(100) not null,
        book_src varchar(200) not null
    )
    """
    cursor.execute(create_table)
    db.commit()

    

    #插入数据
    insert_sql = """
    insert into book1(book_name,book_author,book_company,book_date
    ,book_rank,book_person,book_src) values(%s,%s,%s,%s,%s,%s,%s)
    """

    cursor.executemany(insert_sql,content)
    db.commit()

    db.rollback()

    # try:
    #     for i in content:
    #         cursor.execute(insert_sql,(
    #             i[0],i[1],i[2],i[3],i[4],i[5],i[6]
    #             )
    #         )
    #         db.commit()
            
    # except:
    #     db.rollback()
        
    #4、关闭游标和数据库连接
    cursor.close()
    db.close()

if __name__ == '__main__':
    for i in range(1,13):
        if i == 1:
            url = 'https://book.douban.com/latest?subcat=%E5%85%A8%E9%83%A8&p=1'
        else:
            url = f'https://book.douban.com/latest?subcat=%E5%85%A8%E9%83%A8&p={i}'

        html = get_one_page(url)
        #print(html)     #尝试打印编码 验证是否成功

        content = parse_one_page(html)

        #write_to_csv(content,'Book_data.csv')

        # #写入数据库
        write_to_table(content)




In [28]:
'''
    自行操作 的 豆瓣实训
'''
import csv
import requests
from bs4 import BeautifulSoup
import pymysql
import re

def get_one_page(url):
    header = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0',
        'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'cookie':'ll="118318"; bid=z6gJQ-g_VbA; douban-fav-remind=1; _ga=GA1.1.152072664.1728867251; _ga_RXNMP372GL=GS1.1.1728871022.2.0.1728871022.60.0.0; __utmz=30149280.1729470866.7.6.utmcsr=cn.bing.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmv=30149280.28528; ap_v=0,6.0; _pk_id.100001.3ac3=b11897d958dfba39.1742456607.; _pk_ses.100001.3ac3=1; __utma=30149280.148866276.1726189819.1736752951.1742456607.22; __utma=81379588.152072664.1728867251.1742456607.1742456607.1; __utmz=81379588.1742456607.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); dbcl2="285281789:czKJVYE1R0c"; ck=7OEk; frodotk_db="3c2301308a87d870b46903919f978950"; push_noty_num=0; push_doumail_num=0; __utmc=30149280; __utmt=1; __utmt_douban=1; __utmb=30149280.13.10.1742456607; __utmc=81379588; __utmb=81379588.11.10.1742456607'
    }

    r = requests.get(url,headers = header)
    r.encoding = r.apparent_encoding    #设置编码格式
    demo = r.text
    return demo


def parse_one_page(html):
    soup = BeautifulSoup(html,'html.parser')
    Books = soup.select('#content > div.grid-16-8.clearfix > div.article > ul > li')

    Book_data = []
    for book in Books:
        #书名
        Book_name = book.select('div.media__body > h2 > a')[0].string
        
        #作者 但是带出版社？怎么分割
        Book_author = book.select('div.media__body > p.subject-abstract.color-gray')[0].string.strip().split(' / ')
        
        #作者
        writer = Book_author[0].strip()

        #出版时间
        date = '未知'
        for i in Book_author:
            if re.search(r'\d{4}-\d{1,2}(?:-\d{1,2})?',i):
                date = i.strip()
                break
        

        #出版社
        company = '未知'
        for part in Book_author:
            if re.search(r'.*(?:出版|书店|书局|书馆|编选).*',part):
                company = part.strip()
                break


        '''
        #出版时间
        #date = Book_author[1].strip()   #但是匹配的不准确

        #出版社
        #company = Book_author[2].strip()    #但是匹配的不准确
        '''

        #评分
        Book_elem = book.select('div.media__body > p.clearfix.w250.subject-rating > span.font-small.fleft')
        Book_rank = Book_elem[0].string if Book_elem and Book_elem[0].string else '未评分'
        
        #评分人数
        Book_rankPerson = book.find('span',class_='fleft ml8 color-gray').text.replace('(','').replace(')','')

        #图书链接
        Book_src = book.select('div.media__body > h2 > a')[0]['href']
        

        Book_data.append([Book_name,writer,company,date,Book_rank,Book_rankPerson,Book_src])

        #print(Book_name,writer,company,date,Book_rank,Book_rankPerson,Book_src)
        
        
    return Book_data
        
        


def write_to_csv(content,path):
    with open(path,'a',newline='',encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            writer.writerow(['书名','作者','出版社','出版时间','评分','评分人数','图书链接'])
        writer.writerows(content)   #写入数据


def write_to_table(content):
    #1、创建数据库链接对象connect
    db = pymysql.connect(
        host='localhost',
        port=3306,
        user='root',
        passwd='1024',
        db='douban_book',
        charset='utf8'
    )

    #2、获取游标对象 cursor
    cursor = db.cursor()

    #3、执行SQL语句（核心步骤：建表、增删改查）
    '''

    create_table = """
    create table if not exists book(
        book_name varchar(100) not null primary key,
        book_author varchar(100) not null,
        book_company varchar(100) not null,
        book_date varchar(100) not null,
        book_rank varchar(100) not null,
        book_person varchar(100) not null,
        book_src varchar(200) not null
    )
    """
    cursor.execute(create_table)
    db.commit()

    '''

    #插入数据
    insert_sql = """
    insert into book(book_name,book_author,book_company,book_date
    ,book_rank,book_person,book_src) values(%s,%s,%s,%s,%s,%s,%s)
    """

    # cursor.executemany(insert_sql,content)
    # db.commit()

    # db.rollback()

    try:
        for i in content:
            cursor.execute(insert_sql,(
                i[0],i[1],i[2],i[3],i[4],i[5],i[6]
                )
            )
            db.commit()
            
    except:
        db.rollback()
        
    #4、关闭游标和数据库连接
    cursor.close()
    db.close()

if __name__ == '__main__':
    all_books = []  # 新增：用于收集所有页面的数据
    
    for i in range(1, 13):
        if i == 1:
            url = 'https://book.douban.com/latest?subcat=%E5%85%A8%E9%83%A8&p=1'
        else:
            url = f'https://book.douban.com/latest?subcat=%E5%85%A8%E9%83%A8&p={i}'

        html = get_one_page(url)
        page_books = parse_one_page(html)  # 获取当前页数据
        all_books.extend(page_books)  # 添加到总列表

        print(f"第 {i} 页爬取完成，共 {len(page_books)} 条数据")
    
    # 所有页面爬取完成后，一次性写入
    write_to_csv(all_books, 'Book_data.csv')  # 如果需要写入CSV
    write_to_table(all_books)  # 写入数据库
    
    print(f"全部完成！共爬取 {len(all_books)} 条数据")




第 1 页爬取完成，共 20 条数据
第 2 页爬取完成，共 20 条数据
第 3 页爬取完成，共 20 条数据
第 4 页爬取完成，共 20 条数据
第 5 页爬取完成，共 20 条数据
第 6 页爬取完成，共 20 条数据
第 7 页爬取完成，共 20 条数据
第 8 页爬取完成，共 20 条数据
第 9 页爬取完成，共 20 条数据
第 10 页爬取完成，共 20 条数据
第 11 页爬取完成，共 13 条数据
第 12 页爬取完成，共 0 条数据
全部完成！共爬取 213 条数据
